In [1]:
from Evaluation_tools import *
import os

# Evaluating Linear models

It is important to know the number of models produced in the gridsearch. In this case, we have 80 models, with indexes ranging from 1 to 80

In [2]:
path_of_script='./script_test.sh'
low_bound=1
high_bound=16
name='LETOR'#name that we gave to the linear models
data_fold='MQ2008'#Folder we are going to inspect... LETOR 4
set_evaluation_script(low_bound,high_bound,Folds=[1,2,3,4,5],
                      path_of_script=path_of_script,
                      data_fold=data_fold,name=name)
os.system(path_of_script)

0

One must provide the same hyper parameters used in the training process (and in the same order), so it is possible to associate a performance with a set of hyperparameters. In the following examples we provide an example gridsearch, with only two values for epochs, learning rate, and momentum. Also we evaluate the influence of the chosen features on the metrics

#### There might be some other hyperparameter combination that produces a better output, Nonetheless, we found that most of the best models found are in this area

In [3]:
name='LETOR'#name that we gave to the linear models
from itertools import product
epochs=[500,1000]
learning_rate=[0.001,0.075]
mon=[0.3,0.5]
act=['sigmoid','linear']

columns=["epochs","learning_rate","momemtum","activation"]
dt_linear=pd.DataFrame(columns=columns)

#this lines changes according to the number of hyperparameters used.
#it is the same order as in the training process
for i in (product(epochs,learning_rate,mon,act)):
    dt_linear=dt_linear.append(dict(zip(columns,i)),ignore_index=True)

metrics_df=get_summary_metrics(low_bound,high_bound,return_test=True,
                               name=name,Folds=[1,2,3,4,5],data_fold=data_fold)
#optional... we could leave the precision as well if we wanted to evaluate it
#metrics_df=metrics_df.drop(['Precision mean 1-10 train',
#                            'Precision mean 1-10 vali',
#                            'Precision mean 1-10 test'],axis=1)
hyper_params=dt_linear.iloc[:metrics_df.shape[0],:]
perf=hyper_params.join(metrics_df, how='outer')

### Linear regression

In [4]:
perf[perf['activation']=='linear'].sort_values(['NDCG mean 1-10 vali'],ascending=False).head(5)

,epochs,learning_rate,momemtum,activation,NDCG mean 1-10 train,NDCG mean 1-10 vali,NDCG mean 1-10 test,Precision mean 1-10 train,Precision mean 1-10 vali,Precision mean 1-10 test
0,1000,0.001,0.3,linear,0.398248,0.408446,0.400052,0.336034,0.341948,0.336762


In [5]:
perf[perf['activation']=='linear'].sort_values(['NDCG mean 1-10 test'],ascending=False).head(5)

,epochs,learning_rate,momemtum,activation,NDCG mean 1-10 train,NDCG mean 1-10 vali,NDCG mean 1-10 test,Precision mean 1-10 train,Precision mean 1-10 vali,Precision mean 1-10 test
0,1000,0.001,0.3,linear,0.398248,0.408446,0.400052,0.336034,0.341948,0.336762


### Logistic regression

In [6]:
perf[perf['activation']=='sigmoid'].sort_values(['NDCG mean 1-10 vali'],ascending=False).head(5)

,epochs,learning_rate,momemtum,activation,NDCG mean 1-10 train,NDCG mean 1-10 vali,NDCG mean 1-10 test,Precision mean 1-10 train,Precision mean 1-10 vali,Precision mean 1-10 test


In [7]:
perf[perf['activation']=='sigmoid'].sort_values(['NDCG mean 1-10 test'],ascending=False).head(5)

,epochs,learning_rate,momemtum,activation,NDCG mean 1-10 train,NDCG mean 1-10 vali,NDCG mean 1-10 test,Precision mean 1-10 train,Precision mean 1-10 vali,Precision mean 1-10 test


# Evaluating NN

In [8]:
path_of_script='./script_test.sh'
low_bound=1
#high_bound=16
high_bound=1
name='LETOR_NN'
data_fold='MQ2008'
set_evaluation_script(low_bound,high_bound,Folds=[1,2,3,4,5],
                      path_of_script=path_of_script,
                      data_fold=data_fold,name=name)
os.system(path_of_script)

0

In [9]:
from itertools import product
epochs=[300,400,500,600]
learning_rate=[0.0025]
mon=[0.5]
act=['linear']
inner_act=['relu']
number_layers=[1,2]
number_neurons=[100]
dropout=[0.65,0.8]

columns=["epochs","learning_rate","momemtum","activation",
         "inner_act","number_layers","number_neurons","dropout"]
dt_NN=pd.DataFrame(columns=columns)

for i in (product(epochs,learning_rate,mon,act,inner_act,
                  number_layers,number_neurons,dropout)):
    dt_NN=dt_NN.append(dict(zip(columns,i)),ignore_index=True)

metrics_df=get_summary_metrics(low_bound,high_bound,return_test=True,
                               name=name,Folds=[1,2,3,4,5],data_fold=data_fold)
#optional... we could leave the precision as well if we wanted to evaluate it
#metrics_df=metrics_df.drop(['Precision mean 1-10 train',
#                            'Precision mean 1-10 vali',
#                            'Precision mean 1-10 test'],axis=1)
hyper_params=dt_NN.iloc[:metrics_df.shape[0],:]
perf=hyper_params.join(metrics_df, how='outer')

In [10]:
perf.sort_values(['NDCG mean 1-10 vali'],ascending=False).head(10)

,epochs,learning_rate,momemtum,activation,inner_act,number_layers,number_neurons,dropout,NDCG mean 1-10 train,NDCG mean 1-10 vali,NDCG mean 1-10 test,Precision mean 1-10 train,Precision mean 1-10 vali,Precision mean 1-10 test
0,600,0.0025,0.5,linear,relu,1,100,0.65,0.401006,0.447906,0.398256,0.338226,0.358142,0.338752


**Very close to the baseline, still not far away from the performance obtained with linear models**

# Converting results to CSV

In [11]:
dt = pd.concat([dt_linear,dt_NN], ignore_index=True)[columns]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [12]:
write_metrics_to_csv(low_bound,high_bound,return_test=True,
                     names=['LETOR','LETOR_NN'],Folds=[1,2,3,4,5],data_fold=data_fold,
                     model_dataframe = dt)